In [8]:
import os
import sys
import glob
import pandas as pd
import numpy as np
import xarray as xr
import datetime
from datetime import datetime,timedelta
import matplotlib.pyplot as plt
from itertools import groupby
import statistics

In [2]:

def estimate_lwdown(tair, rh):
    """
    Synthesises downward longwave radiation based on Tair RH

    Params:
    -------
    tair : float
        deg C
    rh : float
        [0-1]

    Reference:
    ----------
    * Abramowitz et al. (2012), Geophysical Research Letters, 39, L04808

    """
    zeroC = 273.15

    sat_vapress = 611.2 * np.exp(17.67 * ((tair - zeroC) / (tair - 29.65)))
    vapress = np.maximum(0.05, rh) * sat_vapress
    lw_down = 2.648 * tair + 0.0346 * vapress - 474.0

    return lw_down

def vpd_to_qair(vpd, tair, press):

    KPA_TO_PA = 1000.
    HPA_TO_PA = 100.0

    tc = tair - 273.15
    # saturation vapor pressure (Pa)
    es = 611.2 * np.exp((17.67 * tc) / (243.5 + tc))

    # vapor pressure
    ea = es - (vpd * HPA_TO_PA)

    qair = 0.622 * ea / (press - (1 - 0.622) * ea)

    return qair


In [3]:


    forecast_date = "2021-01-01"
    siteID = "SRER"
    fname = "data/AmeriFlux/raw/AMF_US-SRC_BASE-BADM_6-5/AMF_US-SRC_BASE_HH_6-5.csv"

    # Open AmeriFlux data
    df = pd.read_csv(fname,comment='#',na_values=-9999)
    # rename columns
    df = df.rename(columns={'TIMESTAMP_START':'dates',
                            'TA':'tair',
                            'RH':'rh',
                            'SW_IN':'swdown',
                            'WS':'wind',
                            'P':'rainf',
                            'VPD_PI':'vpd',
                            'CO2':'co2'})

    # Clean up the dates
    df['dates'] = df['dates'].astype(str)
    new_dates = []
    for i in range(len(df)):
        year = df['dates'][i][0:4]
        month = df['dates'][i][4:6]
        day = df['dates'][i][6:8]
        hour = df['dates'][i][8:10]
        minute = df['dates'][i][10:12]
        if day.startswith("0"):
            day = day[1:]
        if hour.startswith("0"):
            hour = hour[1:]
        date = "%s/%s/%s %s:%s" % (year, month, day, hour, minute)
        new_dates.append(date)

    # re-index df
    df['dates'] = new_dates
    df = df.set_index('dates')
    df.index = pd.to_datetime(df.index)

    # fix units
    kpa_2_pa = 1000.
    deg_2_kelvin = 273.15
    df.tair += deg_2_kelvin
    df.rainf /= 1800. # kg m-2 s-1

    # sort out bad values
    df.swdown = np.where(df.swdown < 0.0, 0.0, df.swdown)
    df.vpd = np.where(df.vpd <= 0.05, 0.05, df.vpd)
    df.rainf = np.where(df.rainf <= 0, 0, df.rainf)





In [4]:
df

,TIMESTAMP_END,USTAR,tair,WD,wind,FC_1_1_1,H,LE,G,TS,...,co2,vpd,SWC_1_1_1,SWC_1_2_1,NETRAD,swdown,SW_OUT,LW_IN,LW_OUT,H2O
dates,,,,,,,,,,,,,,,,,,,,,
2008-01-01 00:00:00,200801010030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01 00:30:00,200801010100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01 01:00:00,200801010130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01 01:30:00,200801010200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01 02:00:00,200801010230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-31 21:30:00,201412312200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-12-31 22:00:00,201412312230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-12-31 22:30:00,201412312300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:

    # Calculate the mean climatology from the data
    meandf = df.groupby([df.index.month, df.index.day]).mean()
    # Put this into imaginary year 2000 (a leap year)
    new_dates = []
    for i in range(len(meandf.index)):
        month = meandf.index[i][0]
        day = meandf.index[i][1]
        date = "2000-%s-%s" % (month, day)
        date = datetime.strptime(date, "%Y-%m-%d")
        new_dates.append(date)

    meandf['date'] = new_dates
    meandf = meandf.set_index('date')




In [7]:
    # Create an index of fake dates
    fakedates = pd.date_range(start="2015-01-01",end=forecast_date,freq='1H', closed='left')
    # Define new dataframe
    newdf = pd.DataFrame({"time" : fakedates,
                     "tair" : np.NaN,
                     "rh" : np.NaN,
                     "swdown" : np.NaN,
                     "wind" : np.NaN,
                     "rainf" : np.NaN,
                     "vpd" : np.NaN,
                     "co2" : np.NaN})

    # Fill new dataframe with averages
    for var in ['swdown','tair','rh', 'wind', 'rainf', 'vpd', 'co2']:
        for i in range(len(newdf)):
            month = newdf['time'][i].month
            day = newdf['time'][i].day
            raw = meandf[var][np.logical_and(meandf.index.month == month , meandf.index.day == day)]
            value = raw.values
            newdf.loc[i, var] = value

    # Add pressure
    newdf['psurf'] = 101325

    # Add LW
    newdf['lwdown'] = estimate_lwdown(newdf.tair.values, newdf.rh.values/100.)

    # Add qair
    newdf['qair'] = vpd_to_qair(newdf.vpd.values, newdf.tair.values, newdf.psurf.values)

    # Remove some rainfall values
    newdf.rainf = np.where(newdf.rainf <= 0.00001, 0, newdf.rainf)


In [6]:
meandf

,TIMESTAMP_END,USTAR,tair,WD,wind,FC_1_1_1,H,LE,G,TS,...,co2,vpd,SWC_1_1_1,SWC_1_2_1,NETRAD,swdown,SW_OUT,LW_IN,LW_OUT,H2O
date,,,,,,,,,,,,,,,,,,,,,
2000-01-01,2.011010e+11,0.172475,280.141008,160.862225,1.606033,-0.251077,25.829298,11.883953,-8.827667,7.492517,...,380.453251,6.594332,9.943592,11.032754,44.292846,163.001925,29.513504,257.084437,344.243358,5.979170
2000-01-02,2.011010e+11,0.173646,282.237254,179.781754,1.643029,-0.173519,23.413658,10.046966,-3.312662,8.490604,...,380.055131,7.814368,9.793921,11.028146,48.376096,164.401903,29.899104,269.135625,352.785538,6.458895
2000-01-03,2.011010e+11,0.227500,282.498912,163.518483,1.980683,-0.180688,26.892477,9.623527,-3.148233,9.266021,...,379.441540,7.594667,9.736742,11.021154,49.630338,163.531317,30.130146,272.599558,355.254208,6.971895
2000-01-04,2.011010e+11,0.173717,281.876683,177.023858,1.552854,-0.246903,21.186528,7.304889,-6.652046,9.278108,...,380.296245,6.608750,9.798550,10.969000,35.334217,126.910221,23.690858,286.916646,353.852221,7.015745
2000-01-05,2.011011e+11,0.157704,280.691771,182.491217,1.534096,-0.249585,23.547764,8.393675,-8.163267,8.858425,...,381.436533,5.781548,10.110992,11.350688,40.847433,138.173273,25.277562,279.710717,349.529433,6.886165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2000-12-27,2.011123e+11,0.211575,279.566071,147.708825,1.791404,-0.152925,20.566059,12.022581,-13.732433,7.665733,...,377.272495,6.297448,10.460804,11.760308,43.451750,135.533500,25.531644,277.719654,343.208500,6.563065
2000-12-28,2.011123e+11,0.187537,280.186804,178.204475,1.597096,-0.189075,18.897495,10.238242,-9.758037,7.859179,...,377.433398,5.228352,10.054292,11.489325,34.671687,134.754921,24.644900,270.730525,345.098792,6.869952
2000-12-29,2.011123e+11,0.238221,282.000746,177.465671,2.023688,-0.193638,22.482574,9.698181,-6.381700,8.564000,...,376.427771,5.832340,9.947900,11.342264,33.917233,127.765473,23.412887,282.787325,351.669758,7.270016


In [8]:
meandf.rainf

date
2000-01-01    0.000000
2000-01-02    0.000000
2000-01-03    0.000000
2000-01-04    0.000019
2000-01-05    0.000002
                ...   
2000-12-27    0.000004
2000-12-28    0.000000
2000-12-29    0.000006
2000-12-30    0.000012
2000-12-31    0.000006
Name: rainf, Length: 366, dtype: float64

In [13]:
    # test generic rainfall
    meandf.rainf = np.where(meandf.rainf > 0.00001, meandf.rainf+0.002, meandf.rainf)

In [14]:
meandf.rainf

date
2000-01-01    0.000000
2000-01-02    0.000000
2000-01-03    0.000000
2000-01-04    0.004019
2000-01-05    0.000002
                ...   
2000-12-27    0.000004
2000-12-28    0.000000
2000-12-29    0.000006
2000-12-30    0.004012
2000-12-31    0.000006
Name: rainf, Length: 366, dtype: float64

In [ ]:
vars_to_keep = ["swdown","vpd","rainf"]
for var in vars_to_keep:
    print("Checking NaN locations for variable ", var)
    for k,g in groupby(meandf[var].isnull().values):
       print((k, sum(1 for i in g)))

In [13]:
forecast_date = datetime.strptime("2021-02-01",'%Y-%m-%d')

In [12]:
forecast_date - timedelta(days=1)

datetime.datetime(2021, 1, 31, 0, 0)

In [16]:
datetime.strftime(forecast_date,'%Y-%m-%d')

'2021-02-01'

In [17]:
"site is"+forecast_date

TypeError: can only concatenate str (not "datetime.datetime") to str